### Building out a neural network

In [3]:
from micrograd.engine import Value
import numpy as np

* The constructor __init__ takes in number of inputs into the neuron as an argument. It creates a weight for each input, which is a random number between -1 and 1. It also creates a bias (controlling the over all "trigger-happiness" of the neuron) in the same range.
* __call__ returns an output of the forward pass. We zip together w1 and x1 and multiply them together pairwise. We sum the results and add the bias (added as the second argument in the sum function). Lastly, we pass this through an activation function - in this case relu. Re-running the cell returns a different output each time as the weights and biases are randomly initialised each time.

So, below is just one neuron. 

In [23]:
class Neuron:

    def __init__(self, nin):
        self.w = [Value(np.random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(np.random.uniform(-1,1))

    def __call__(self, x):
        # w * x + b. 
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.relu()
        return out

x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(data=2.03800872160737, grad=0)

A layer is made of multiple neurons. The neurons in that layer are not connected to each other, but are connected to each of the inputs. We are defining a Layer class, which is literally just a list of neurons.
* In the constructor, nin is number of neurons in the layer, nout is the number of layers.

In [26]:
class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
x = [2.0, 3.0]
n = Layer(2, 3)
n(x)

[Value(data=0, grad=0),
 Value(data=0, grad=0),
 Value(data=4.783731524406042, grad=0)]

#### An MLP

In [19]:
class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [27]:
# So if we want to implement a network with 3 input neurons, 2 layers of 4 neurons and 1 output neuron:

inputs = [2.0, 3.0, -1.0]
n = MLP(3, [4,4,1])
n(x)

# And boom, there is a forward pass through the network.

Value(data=1.4670991636055768, grad=0)

#### Another example

Create a simple dataset. There are 4 examples with 3 inputs for each. We also have a list of desired targets. So we want the weights to be adjusted so that the first output = 1.0, etc. 

In [30]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [33]:
ypred = [n(x) for x in xs]
ypred

[Value(data=2.031693635563312, grad=0),
 Value(data=0.7530379027357996, grad=0),
 Value(data=0, grad=0),
 Value(data=1.2902091746378512, grad=0)]

They each have a way to go. We'd like to push all 4 down to reach their respective targets. We do this by implementing a loss function.

In [34]:
# Mean squared error
[(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]

[Value(data=1.064391757661844, grad=0),
 Value(data=3.073141888428331, grad=0),
 Value(data=1.0, grad=0),
 Value(data=0.08422136504398284, grad=0)]

They've been nudged.